# Chapter 23 - Databases and SQL

The data you need will often live in databases, systems designed for efficiently storing
and querying data. The bulk of these are relational databases, such as Oracle, MySQL,
and SQL Server, which store data in tables and are typically queried using Structured
Query Language (SQL), a declarative language for manipulating data.

SQL is a pretty essential part of the data scientist’s toolkit. In this chapter, *__we’ll create
NotQuiteABase, a Python implementation of something that’s not quite a database__*.
We’ll also cover the basics of SQL while showing how they work in our not-quite
database, which is the most “from scratch” way I could think of to help you under‐
stand what they’re doing. My hope is that *__solving problems in NotQuiteABase will
give you a good sense of how you might solve the same problems using SQL__*.

## CREATE TABLE and INSERT

A relational database is a collection of tables (and of relationships among them). A
table is simply a collection of rows, not unlike the matrices we’ve been working with.
However, a table also has associated with it a fixed schema consisting of column
names and column types.
For example, imagine a users data set containing for each user her user_id, name,
and num_friends:

In [1]:
users = [[0, "Hero", 0],
         [1, "Dunn", 2],
         [2, "Sue", 3],
         [3, "Chi", 3]]

In [2]:
users

[[0, 'Hero', 0], [1, 'Dunn', 2], [2, 'Sue', 3], [3, 'Chi', 3]]

In SQL, we might create this table with:

    CREATE TABLE users (
        user_id INT NOT NULL,
        name VARCHAR(200),
        num_friends INT);
        
Notice that we specified that the user_id and num_friends must be integers (and that
user_id isn’t allowed to be NULL, which indicates a missing value and is sort of like
our None) and that the name should be a string of length 200 or less. NotQuiteABase
won’t take types into account, but we’ll behave as if it did.

You can insert the rows with INSERT statements:

    INSERT INTO users (user_id, name, num_friends) VALUES (0, 'Hero', 0);
    
In NotQuiteABase, you’ll create a Table simply by specifying the names of its col‐
umns. And to insert a row, you’ll use the table’s insert() method, which takes a list
of row values that need to be in the same order as the table’s column names.

Behind the scenes, we’ll store each row as a dict from column names to values. A real
database would never use such a space-wasting representation, but doing so will
make NotQuiteABase much easier to work with:

In [3]:
class Table:
    def __init__(self, columns):
        self.columns = columns
        self.rows = []
        
    def __repr__(self):
        """Pretty representation of the table: columns then rows."""
        return str(self.columns) + "\n" + "\n".join(map(str,self.rows))
    
    def insert(self, row_values):
        if len(row_values) != len(self.columns):
            raise TypeError("wrong number of elements")
        row_dict = dict(zip(self.columns,row_values))
        self.rows.append(row_dict)

In [4]:
users = Table(["user_id", "name", "num_friends"])

In [5]:
users

['user_id', 'name', 'num_friends']

In [6]:
users.insert([0, "Hero", 0])

In [7]:
users

['user_id', 'name', 'num_friends']
{'user_id': 0, 'name': 'Hero', 'num_friends': 0}

In [8]:
users.insert([1, "Dunn", 2])
users.insert([2, "Sue", 3])
users.insert([3, "Chi", 3])
users.insert([4, "Thor", 3])
users.insert([5, "Clive", 2])
users.insert([6, "Hicks", 3])
users.insert([7, "Devin", 2])
users.insert([8, "Kate", 2])
users.insert([9, "Klein", 3])
users.insert([10, "Jen", 1])

If you now print users, you’ll see:

In [10]:
users

['user_id', 'name', 'num_friends']
{'user_id': 0, 'name': 'Hero', 'num_friends': 0}
{'user_id': 1, 'name': 'Dunn', 'num_friends': 2}
{'user_id': 2, 'name': 'Sue', 'num_friends': 3}
{'user_id': 3, 'name': 'Chi', 'num_friends': 3}
{'user_id': 4, 'name': 'Thor', 'num_friends': 3}
{'user_id': 5, 'name': 'Clive', 'num_friends': 2}
{'user_id': 6, 'name': 'Hicks', 'num_friends': 3}
{'user_id': 7, 'name': 'Devin', 'num_friends': 2}
{'user_id': 8, 'name': 'Kate', 'num_friends': 2}
{'user_id': 9, 'name': 'Klein', 'num_friends': 3}
{'user_id': 10, 'name': 'Jen', 'num_friends': 1}

## UPDATE

Sometimes you need to update the data that’s already in the database. For instance, if
Dunn acquires another friend, you might need to do this:

    UPDATE users
    SET num_friends = 3
    WHERE user_id = 1;
    
The key features are:
* What table to update
* Which rows to update
* Which fields to update
* What their new values should be

We’ll add a similar update method to NotQuiteABase. *__Its first argument will be a
dict whose keys are the columns to update and whose values are the new values for
those fields. And its second argument is a predicate that returns True for rows that
should be updated__*, False otherwise:

In [26]:
class Table:
    def __init__(self, columns):
        self.columns = columns
        self.rows = []
        
    def __repr__(self):
        """Pretty representation of the table: columns then rows."""
        return str(self.columns) + "\n" + "\n".join(map(str,self.rows))
    
    def insert(self, row_values):
        if len(row_values) != len(self.columns):
            raise TypeError("wrong number of elements")
        row_dict = dict(zip(self.columns,row_values))
        self.rows.append(row_dict)

    def update(self, updates, predicate):
        for row in self.rows:
            if predicate(row):
                for column, new_value in updates.items():
                    row[column] = new_value

after which we can simply do this:

In [41]:
users = Table(["user_id", "name", "num_friends"])

users.insert([0, "Hero", 0])
users.insert([1, "Dunn", 2])
users.insert([2, "Sue", 3])
users.insert([3, "Chi", 3])
users.insert([4, "Thor", 3])
users.insert([5, "Clive", 2])
users.insert([6, "Hicks", 3])
users.insert([7, "Devin", 2])
users.insert([8, "Kate", 2])
users.insert([9, "Klein", 3])
users.insert([10, "Jen", 1])

In [42]:
users

['user_id', 'name', 'num_friends']
{'user_id': 0, 'name': 'Hero', 'num_friends': 0}
{'user_id': 1, 'name': 'Dunn', 'num_friends': 2}
{'user_id': 2, 'name': 'Sue', 'num_friends': 3}
{'user_id': 3, 'name': 'Chi', 'num_friends': 3}
{'user_id': 4, 'name': 'Thor', 'num_friends': 3}
{'user_id': 5, 'name': 'Clive', 'num_friends': 2}
{'user_id': 6, 'name': 'Hicks', 'num_friends': 3}
{'user_id': 7, 'name': 'Devin', 'num_friends': 2}
{'user_id': 8, 'name': 'Kate', 'num_friends': 2}
{'user_id': 9, 'name': 'Klein', 'num_friends': 3}
{'user_id': 10, 'name': 'Jen', 'num_friends': 1}

In [30]:
users.update({'num_friends' : 3}, # set num_friends = 3
    lambda row: row['user_id'] == 1) # in rows where user_id == 1

In [31]:
users

['user_id', 'name', 'num_friends']
{'user_id': 0, 'name': 'Hero', 'num_friends': 0}
{'user_id': 1, 'name': 'Dunn', 'num_friends': 3}
{'user_id': 2, 'name': 'Sue', 'num_friends': 3}
{'user_id': 3, 'name': 'Chi', 'num_friends': 3}
{'user_id': 4, 'name': 'Thor', 'num_friends': 3}
{'user_id': 5, 'name': 'Clive', 'num_friends': 2}
{'user_id': 6, 'name': 'Hicks', 'num_friends': 3}
{'user_id': 7, 'name': 'Devin', 'num_friends': 2}
{'user_id': 8, 'name': 'Kate', 'num_friends': 2}
{'user_id': 9, 'name': 'Klein', 'num_friends': 3}
{'user_id': 10, 'name': 'Jen', 'num_friends': 1}

## DELETE

There are two ways to delete rows from a table in SQL. The dangerous way deletes
every row from a table:

    DELETE FROM users;
    
The less dangerous way adds a WHERE clause and only deletes rows that match a cer‐
tain condition:

    DELETE FROM users WHERE user_id = 1;
    
It’s easy to add this functionality to our Table:

In [38]:
class Table:
    def __init__(self, columns):
        self.columns = columns
        self.rows = []
        
    def __repr__(self):
        """Pretty representation of the table: columns then rows."""
        return str(self.columns) + "\n" + "\n".join(map(str,self.rows))
    
    def insert(self, row_values):
        if len(row_values) != len(self.columns):
            raise TypeError("wrong number of elements")
        row_dict = dict(zip(self.columns,row_values))
        self.rows.append(row_dict)

    def update(self, updates, predicate):
        for row in self.rows:
            if predicate(row):
                for column, new_value in updates.items():
                    row[column] = new_value

    def delete(self, predicate = lambda row:True):
        """Delete all rows matching predicate
        or all rows if no predicate supplied."""
        self.rows = [row for row in self.rows if not(predicate(row))]

If you supply a predicate function (i.e., a WHERE clause), this deletes only the rows
that satisfy it. If you don’t supply one, the default predicate always returns True, and
you will delete every row.

In [44]:
users = Table(["user_id", "name", "num_friends"])

users.insert([0, "Hero", 0])
users.insert([1, "Dunn", 2])
users.insert([2, "Sue", 3])
users.insert([3, "Chi", 3])
users.insert([4, "Thor", 3])
users.insert([5, "Clive", 2])
users.insert([6, "Hicks", 3])
users.insert([7, "Devin", 2])
users.insert([8, "Kate", 2])
users.insert([9, "Klein", 3])
users.insert([10, "Jen", 1])

In [45]:
users

['user_id', 'name', 'num_friends']
{'user_id': 0, 'name': 'Hero', 'num_friends': 0}
{'user_id': 1, 'name': 'Dunn', 'num_friends': 2}
{'user_id': 2, 'name': 'Sue', 'num_friends': 3}
{'user_id': 3, 'name': 'Chi', 'num_friends': 3}
{'user_id': 4, 'name': 'Thor', 'num_friends': 3}
{'user_id': 5, 'name': 'Clive', 'num_friends': 2}
{'user_id': 6, 'name': 'Hicks', 'num_friends': 3}
{'user_id': 7, 'name': 'Devin', 'num_friends': 2}
{'user_id': 8, 'name': 'Kate', 'num_friends': 2}
{'user_id': 9, 'name': 'Klein', 'num_friends': 3}
{'user_id': 10, 'name': 'Jen', 'num_friends': 1}

In [46]:
users.delete(lambda row: row["user_id"] == 1) # deletes rows with user_id == 1

In [47]:
users

['user_id', 'name', 'num_friends']
{'user_id': 0, 'name': 'Hero', 'num_friends': 0}
{'user_id': 2, 'name': 'Sue', 'num_friends': 3}
{'user_id': 3, 'name': 'Chi', 'num_friends': 3}
{'user_id': 4, 'name': 'Thor', 'num_friends': 3}
{'user_id': 5, 'name': 'Clive', 'num_friends': 2}
{'user_id': 6, 'name': 'Hicks', 'num_friends': 3}
{'user_id': 7, 'name': 'Devin', 'num_friends': 2}
{'user_id': 8, 'name': 'Kate', 'num_friends': 2}
{'user_id': 9, 'name': 'Klein', 'num_friends': 3}
{'user_id': 10, 'name': 'Jen', 'num_friends': 1}

## SELECT

Typically you don’t inspect SQL tables directly. Instead you query them with a SELECT
statement:

    SELECT * FROM users; -- get the entire contents
    SELECT * FROM users LIMIT 2; -- get the first two rows
    SELECT user_id FROM users; -- only get specific columns
    SELECT user_id FROM users WHERE name = 'Dunn'; -- only get specific rows
    
You can also use SELECT statements to calculate fields:
    
    SELECT LENGTH(name) AS name_length FROM users;
    
We’ll give our Table class a select() method that returns a new Table. The method
accepts two optional arguments:
* keep_columns specifies the name of the columns you want to keep in the result.
If you don’t supply it, the result contains all the columns.
* additional_columns is a dictionary whose keys are new column names and
whose values are functions specifying how to compute the values of the new columns.

If you were to supply neither of them, you’d simply get back a copy of the table:

In [49]:
def select(self, keep_columns=None, additional_columns=None):
    if keep_columns is None:        # If no columns specified
        keep_columns = self.columns # return all columns.
        
    if additional_columns is None:
        additional_columns = {}
        
    # new table for results
    result_table = Table(keep_columns + additional_column_keys())
    
    for row in self.rows:
        new_row = [row[column] for column in keep_columns]
        for column_name, calculation in additional_columns.items():
            new_row.append(calculation(row))
        result_table.insert(new_row)
        
    return result_table

Our select() returns a new Table, while the typical SQL SELECT just produces some
sort of transient result set (unless you explicitly insert the results into a table).

We’ll also need where() and limit() methods. Both are pretty simple:

In [50]:
def where(self, predicate=lambda row: True):
    """Return only the rows that satisfy the supplied predicate."""
    where_table = Table(self.columns)
    where_table.rows = filter(predicate, self.rows)
    return where_table

def limit(self, num_rows):
    """Return only the first num_rows rows."""
    limit_table = Table(self.columns)
    limit_table.rows = self.rows[:num_rows]
    return limit_table

after which we can easily construct NotQuiteABase equivalents to the preceding SQL
statements:
    
    # SELECT * FROM users;
    users.select()
    
    # SELECT * FROM users LIMIT 2;
    users.limit(2)
    
    # SELECT user_id FROM users;
    users.select(keep_columns=["user_id"])

In [61]:
class Table:
    def __init__(self, columns):
        self.columns = columns
        self.rows = []
        
    def __repr__(self):
        """Pretty representation of the table: columns then rows."""
        return str(self.columns) + "\n" + "\n".join(map(str,self.rows))
    
    def insert(self, row_values):
        if len(row_values) != len(self.columns):
            raise TypeError("wrong number of elements")
        row_dict = dict(zip(self.columns,row_values))
        self.rows.append(row_dict)

    def update(self, updates, predicate):
        for row in self.rows:
            if predicate(row):
                for column, new_value in updates.items():
                    row[column] = new_value
                    
    def select(self, keep_columns=None, additional_columns=None):
        if keep_columns is None:        # If no columns specified
            keep_columns = self.columns # return all columns.

        if additional_columns is None:
            additional_columns = {}

        # new table for results
        result_table = Table(keep_columns + additional_columns.keys())

        for row in self.rows:
            new_row = [row[column] for column in keep_columns]
            for column_name, calculation in additional_columns.items():
                new_row.append(calculation(row))
            result_table.insert(new_row)

        return result_table

    def where(self, predicate=lambda row: True):
        """Return only the rows that satisfy the supplied predicate."""
        where_table = Table(self.columns)
        where_table.rows = filter(predicate, self.rows)
        return where_table

    def limit(self, num_rows):
        """Return only the first num_rows rows."""
        limit_table = Table(self.columns)
        limit_table.rows = self.rows[:num_rows]
        return limit_table

In [62]:
users = Table(["user_id", "name", "num_friends"])

users.insert([0, "Hero", 0])
users.insert([1, "Dunn", 2])
users.insert([2, "Sue", 3])
users.insert([3, "Chi", 3])
users.insert([4, "Thor", 3])
users.insert([5, "Clive", 2])
users.insert([6, "Hicks", 3])
users.insert([7, "Devin", 2])
users.insert([8, "Kate", 2])
users.insert([9, "Klein", 3])
users.insert([10, "Jen", 1])

In [63]:
users

['user_id', 'name', 'num_friends']
{'user_id': 0, 'name': 'Hero', 'num_friends': 0}
{'user_id': 1, 'name': 'Dunn', 'num_friends': 2}
{'user_id': 2, 'name': 'Sue', 'num_friends': 3}
{'user_id': 3, 'name': 'Chi', 'num_friends': 3}
{'user_id': 4, 'name': 'Thor', 'num_friends': 3}
{'user_id': 5, 'name': 'Clive', 'num_friends': 2}
{'user_id': 6, 'name': 'Hicks', 'num_friends': 3}
{'user_id': 7, 'name': 'Devin', 'num_friends': 2}
{'user_id': 8, 'name': 'Kate', 'num_friends': 2}
{'user_id': 9, 'name': 'Klein', 'num_friends': 3}
{'user_id': 10, 'name': 'Jen', 'num_friends': 1}

In [65]:
# SELECT user_id FROM users WHERE name = 'Dunn';
users.where(lambda row: row["name"] == "Dunn").select(keep_columns=["user_id"])

TypeError: can only concatenate list (not "dict_keys") to list

## GROUP BY

Another common SQL operation is GROUP BY, which groups together rows with identical values in specified columns and produces aggregate values like MIN and MAX and
COUNT and SUM.

For example, you might want to find the number of users and the smallest user_id
for each possible name length:

    SELECT LENGTH(name) as name_length,
        MIN(user_id) AS min_user_id,
        COUNT(*) AS num_users
    FROM users
    GROUP BY LENGTH(name);
    
Every field we SELECT needs to be either in the GROUP BY clause (which name_length
is) or an aggregate computation (which min_user_id and num_users are).

SQL also supports a HAVING clause that behaves similarly to a WHERE clause except that
its filter is applied to the aggregates (whereas a WHERE would filter out rows before
aggregation even took place).

You might want to know the average number of friends for users whose names start
with specific letters but only see the results for letters whose corresponding average is
greater than 1. (Yes, some of these examples are contrived.)

    SELECT SUBSTR(name, 1, 1) AS first_letter,
        AVG(num_friends) AS avg_num_friends
    FROM users
    GROUP BY SUBSTR(name, 1, 1)
    HAVING AVG(num_friends) > 1;
    
You can also compute overall aggregates. In that case, you leave off the GROUP BY:

    SELECT SUM(user_id) as user_id_sum
    FROM users
    WHERE user_id > 1;
    
To add this functionality to NotQuiteABase Tables, we’ll add a group_by() method.
It takes the names of the columns you want to group by, a dictionary of the aggrega‐
tion functions you want to run over each group, and an optional predicate having
that operates on multiple rows.

Then it does the following steps:
1. Creates a defaultdict to map tuples (of the group-by-values) to rows (contain‐
ing the group-by-values). Recall that you can’t use lists as dict keys; you have to
use tuples.
2. Iterates over the rows of the table, populating the defaultdict.
3. Creates a new table with the correct output columns.
4. Iterates over the defaultdict and populates the output table, applying the having filter if any.

In [67]:
class Table:
    def __init__(self, columns):
        self.columns = columns
        self.rows = []
        
    def __repr__(self):
        """Pretty representation of the table: columns then rows."""
        return str(self.columns) + "\n" + "\n".join(map(str,self.rows))
    
    def insert(self, row_values):
        if len(row_values) != len(self.columns):
            raise TypeError("wrong number of elements")
        row_dict = dict(zip(self.columns,row_values))
        self.rows.append(row_dict)

    def update(self, updates, predicate):
        for row in self.rows:
            if predicate(row):
                for column, new_value in updates.items():
                    row[column] = new_value
                    
    def select(self, keep_columns=None, additional_columns=None):
        if keep_columns is None:        # If no columns specified
            keep_columns = self.columns # return all columns.

        if additional_columns is None:
            additional_columns = {}

        # new table for results
        result_table = Table(keep_columns + additional_columns.keys())

        for row in self.rows:
            new_row = [row[column] for column in keep_columns]
            for column_name, calculation in additional_columns.items():
                new_row.append(calculation(row))
            result_table.insert(new_row)

        return result_table

    def where(self, predicate=lambda row: True):
        """Return only the rows that satisfy the supplied predicate."""
        where_table = Table(self.columns)
        where_table.rows = filter(predicate, self.rows)
        return where_table

    def limit(self, num_rows):
        """Return only the first num_rows rows."""
        limit_table = Table(self.columns)
        limit_table.rows = self.rows[:num_rows]
        return limit_table

    def group_by(self, group_by_columns, aggregates, having=None):

        grouped_rows = defaultdict(list)

        #populate groups
        for row in self.rows:
            key = tuple(row[column] for column in group_by_columns)
            grouped_rows[key].append(row)

        # result table consists of group_by columns and aggregates
        result_table = Table(group_by_columns + aggregates.keys())

        for key, rows in grouped_rows.iteritems():
            if having is None or having(rows):
                new_row = list(key)
                for aggregate_name, aggregate_fn in aggregates.iteritems():
                    new_row.append(aggregate_fn(rows))
                result_table.insert(new_row)

        return result_table

In [68]:
users = Table(["user_id", "name", "num_friends"])

users.insert([0, "Hero", 0])
users.insert([1, "Dunn", 2])
users.insert([2, "Sue", 3])
users.insert([3, "Chi", 3])
users.insert([4, "Thor", 3])
users.insert([5, "Clive", 2])
users.insert([6, "Hicks", 3])
users.insert([7, "Devin", 2])
users.insert([8, "Kate", 2])
users.insert([9, "Klein", 3])
users.insert([10, "Jen", 1])

Again, let’s see how we would do the equivalent of the preceding SQL statements. The
name_length metrics are:

In [69]:
def min_user_id(rows): return min(row["user_id"] for row in rows)

In [70]:
stats_by_length = users.select(additional_columns={"name_length" : name_length}) \
                    .group_by(group_by_columns=["name_length"], 
                              aggregates={"min_user_id" : min_user_id, "num_user_id" : len})

NameError: name 'name_length' is not defined